In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv('ecom_processed.csv')
df.head()

,Unnamed: 0,user_id,region,device,channel,session_start,session_end,sessiondurationsec,session_date,month,day,hour_of_day,order_dt,revenue,payment_type,promo_code,payer,time_of_day,final_price
0,0,529697267522,United States,iPhone,социальные сети,2019-05-01 00:06:40,2019-05-01 00:07:06,26,2019-05-01,5,3,0,2019-05-01 00:06:40,9999.0,Mobile payments,0.0,1,ночь,9999.0
1,1,601292388085,United States,PC,organic,2019-05-01 06:56:16,2019-05-01 07:09:18,782,2019-05-01,5,3,7,NaN,NaN,NaN,NaN,0,утро,NaN
2,2,852898876338,United States,Mac,социальные сети,2019-05-01 04:30:45,2019-05-01 04:34:56,251,2019-05-01,5,3,4,NaN,NaN,NaN,NaN,0,ночь,NaN
3,3,998513020664,United States,iPhone,социальные сети,2019-05-01 18:53:42,2019-05-01 18:57:35,233,2019-05-01,5,3,18,NaN,NaN,NaN,NaN,0,вечер,NaN
4,4,240702200943,United States,Mac,социальные сети,2019-05-02 14:04:32,2019-05-02 14:09:51,319,2019-05-02,5,4,14,NaN,NaN,NaN,NaN,0,день,NaN


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997 entries, 0 to 996
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          997 non-null    int64  
 1   user_id             997 non-null    int64  
 2   region              997 non-null    object 
 3   device              997 non-null    object 
 4   channel             997 non-null    object 
 5   session_start       997 non-null    object 
 6   session_end         997 non-null    object 
 7   sessiondurationsec  997 non-null    int64  
 8   session_date        997 non-null    object 
 9   month               997 non-null    int64  
 10  day                 997 non-null    int64  
 11  hour_of_day         997 non-null    int64  
 12  order_dt            275 non-null    object 
 13  revenue             275 non-null    float64
 14  payment_type        275 non-null    object 
 15  promo_code          275 non-null    float64
 16  payer   

Средний чек

In [17]:
df.final_price.mean().round(2)

5416.09

Среднее кол-во покупок от пользователя

In [18]:
df.payer.mean().round(2)

0.28

Средняя продолжительность сессии по каналам рекламы

In [19]:
df.groupby('channel').sessiondurationsec.mean().round(2)

channel
email-рассылки         1919.26
organic                1725.96
контекстная реклама    1871.79
реклама у блогеров     1669.64
социальные сети        1602.86
Name: sessiondurationsec, dtype: float64

Средняя продолжительность сессии по типу устройства

In [20]:
df.groupby('device').sessiondurationsec.mean().round(2)

device
Android    1801.14
Mac        1616.55
PC         1806.77
iPhone     1625.27
Name: sessiondurationsec, dtype: float64

Топ-3 рекламных канала по среднему чеку

In [21]:
df.groupby('channel').final_price.mean().sort_values(ascending=False).round(2).head(3)

channel
социальные сети       5525.13
organic               5433.48
реклама у блогеров    5388.66
Name: final_price, dtype: float64

Топ-3 региона по среднему чеку

In [22]:
df.groupby('region').final_price.mean().sort_values(ascending=False).round(2).head(3)

region
UK               5555.25
United States    5423.23
Germany          5344.83
Name: final_price, dtype: float64

Топ-3 месяца по среднему чеку с разбивкой по регионам

In [23]:
def func_month(x):
    if x == 5:
        return "May"
    if x == 6:
        return "June"
    if x == 7:
        return "July"
    if x == 8:
        return "August"
    if x == 9:
        return "September"
    if x == 10:
        return "October"


df['month'] = pd.DatetimeIndex(df['session_date']).month
df['month'] = df['month'].apply(func_month)

0          May
1          May
2          May
3          May
4          May
        ...   
992    October
993    October
994    October
995    October
996    October
Name: month, Length: 997, dtype: object

In [36]:
reg_df = pd.DataFrame()
reg_df['region'] = ['United States'] * 6 + ['France'] * 6 + ['Germany'] * 6 + ['UK'] * 6
reg_df['month'] = ['May', 'June', 'July', 'August', 'September', 'October'] * 4
reg_df['mean_price'] = list(df.groupby(['region', 'month']).final_price.mean().round(2))
reg_df

,region,month,mean_price
0,United States,May,4999.00
1,United States,June,4999.00
2,United States,July,5499.00
3,United States,August,5449.00
4,United States,September,5165.67
5,United States,October,4999.00
6,France,May,4987.89
7,France,June,7499.00
8,France,July,5679.00
9,France,August,4665.67


In [44]:
reg_df.groupby(['region', 'month']).agg({'mean_price':sum})['mean_price'].groupby('region', group_keys=False).apply(lambda x : x.sort_values(ascending=False).head(3))

C:\Users\did\AppData\Local\Temp\ipykernel_280\2883707028.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  reg_df.groupby(['region', 'month']).agg({'mean_price':sum})['mean_price'].groupby('region', group_keys=False).apply(lambda x : x.sort_values(ascending=False).head(3))


region         month    
France         June         7499.00
               July         5679.00
               October      5249.00
Germany        September    6056.14
               August       5999.00
               May          5610.11
UK             August       6051.17
               May          5567.09
               July         5459.00
United States  July         5499.00
               August       5449.00
               September    5165.67
Name: mean_price, dtype: float64

MAU* по каждому месяцу с разбивкой по рекламным каналам\
*MAU (Monthly Active Users) — это количество уникальных пользователей за месяц.